In [1]:
import copy
import cv2
import datetime
import glob
import h5py
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy
import scipy.signal
import skimage
import sys
import tifffile
import time
from pathlib import Path
from lbm_util import init_params
plt.rcParams["figure.dpi"] = 900

try:
    from icecream import ic, install, argumentToString
    install()
except ImportError:  # graceful fallback if icecream isn't installed.
    ic = lambda *a: None if not a else (a[0] if len(a) == 1 else a)  # noqa

@argumentToString.register(np.ndarray)
def _(obj):
    """Format our return for the icecream debug statement."""
    return f"ndarray, shape={obj.shape}, dtype={obj.dtype}"

params = init_params()

if params['debug']:
    ic.enable()
    ic.configureOutput(prefix='RBO Debugger -> ', includeContext=True, contextAbsPath=True)
    ic()
else:
    ic.disable()
    ic()


In [2]:
if params["save_output"]:
    # 'planes' will save individual planes in subfolders -- 'volume' will save a whole 4D hdf5 volume
    params["save_as_volume_or_planes"] = "planes"
    if params["save_as_volume_or_planes"] == "planes":
        # If True, it will take all the time-chunked h5 files, concatenate, and save them as a single .tif
        params["concatenate_all_h5_to_tif"] = False

if params["seams_overlap"] == "calculate":
    # correct delay or incorrect phase when EOM turns the laser on/off at the start/end of a resonant-scanner line
    params["n_ignored_pixels_sides"] = 5
    params["min_seam_overlap"] = 5
    params["max_seam_overlap"] = 20
    params["alignment_plot_checks"] = False
if not params["reconstruct_all_files"]:
    params["reconstruct_until_this_ifile"] = 10
if params["save_mp4"] or params["save_meanf_png"]:
    params["gaps_columns"] = 5
    params["gaps_rows"] = 5
    params["intensity_percentiles"] = [15, 99.5]
    if params["save_meanf_png"]:
        params["meanf_png_only_first_file"] = True
    if params["save_mp4"]:
        params["video_only_first_file"] = True
        params["video_play_speed"] = 1
        params["rolling_average_frames"] = 1
        params["video_duration_secs"] = 20
        
if not params["lateral_align_planes"]:
    initialize_volume_with_nans = False
    convert_volume_float32_to_int16 = True
    params["make_nonan_volume"] = False
elif params["make_nonan_volume"]:
    initialize_volume_with_nans = True
    convert_volume_float32_to_int16 = True
else:
    initialize_volume_with_nans = True
    convert_volume_float32_to_int16 = False

NameError: name 'params' is not defined

In [ ]:
# %% Look for files used to: 1) make a template and do seam-overlap handling and X-Y shift alignment; 2) pre-process
path_all_files = []
for i_dir in params["raw_data_dirs"]:
    tmp_paths = sorted(glob.glob(i_dir + "/**/*.tif", recursive=True))
    for this_tmp_path in tmp_paths:
        if (
                params["fname_must_contain"] in this_tmp_path and
                params["fname_must_NOT_contain"] not in this_tmp_path
        ):
            path_all_files.append(this_tmp_path)

if params["debug"]:
    ic(path_all_files)

n_template_files = len(params["list_files_for_template"])
ic(n_template_files)
path_template_files = [path_all_files[file_idx] for file_idx in params["list_files_for_template"]]

del (
    i_dir,
    params["raw_data_dirs"],
    params["fname_must_contain"],
    params["fname_must_NOT_contain"],
)

pipeline_steps = []
if params["make_template_seams_and_plane_alignment"]:
    pipeline_steps.append("make_template")
if params["reconstruct_all_files"]:
    pipeline_steps.append("reconstruct_all")
